In [1]:
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [139]:
#計測地点のosm_idを指定
start_osm = 310280534
mid_osm = []
end_osm = 310280538
start_mesh = 50325671212
end_mesh = 50325662413

In [140]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [141]:

sttime = "2022-04-01 12:00:00"
dbname = "kddi_trkn12_change_a26"

In [142]:
datedir = "230213"
f = open(datedir+'/'+ dbname + '.csv', 'w')

cur =connection.cursor()
sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
cur.execute(sql)
for row in cur:
    stlat = row[0]
    stlng = row[1]
sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
cur.execute(sql)
for row in cur:
    edlat = row[0]
    edlng = row[1]
cur.close()

if stlat > edlat:
    wklat = stlat
    stlat = edlat
    edlat = wklat
if stlng > edlng:
    wklng = stlng
    stlng = edlng
    edlng = wklng
            
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM kddi_12_param  "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)


        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        sql += "ORDER BY trakanmap.gridcode"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if dis_km == 0:
                    ret = 0
                elif trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                #cur.execute(sql)
                f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + '\n')
                #connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        f.write(str(dte) +  "," + str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + '\n')
        #cur.execute(sql)
        #connection.commit()
                
        cur.close()
        #print(lstItems)
        dte = dte + datetime.timedelta(minutes=15)
f.close()

33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値55.0距離0.0 km
50325671221
存在メッシュ：50325671221値75.54390907497996距離0.14467375091535445 km
50325661434
存在メッシュ：50325661434値71.4069444299371距離0.11554053238239893 km
50325672211
存在メッシュ：50325672211値198.80736345787517距離1.0127162559357643 km
50325672122
存在メッシュ：50325672122値178.26345440810485距離0.8680425051979405 km
50325671222
存在メッシュ：50325671222値96.08781814841464距離0.2893475018198268 km
50325671221
存在メッシュ：50325671221値75.54390907497996距離0.14467375091535445 km
50325672111
存在メッシュ：50325672111値116.6317272198799距離0.4340212527104301 km
50325672121
存在メッシュ：50325672121値157.7195453508654距離0.7233687544075176 km
50325672112
存在メッシュ：50325672112値137.17563628783043距離0.578695003576282 km
50325671212
存在メッシュ：50325671212値55.0距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値50.0距離0.0 km
50325671221
存在メッシュ：50325671221値72.97674041280654距離0.14467375091535445 km
50325661434
存在メッシュ：50325661434値

33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値54.0距離0.0 km
50325671221
存在メッシュ：50325671221値74.40875177843404距離0.14467375091535445 km
50325661434
存在メッシュ：50325661434値70.29900400605592距離0.11554053238239893 km
50325672211
存在メッシュ：50325672211値196.8612623824944距離1.0127162559357643 km
50325672122
存在メッシュ：50325672122値176.45251062910415距離0.8680425051979405 km
50325671222
存在メッシュ：50325671222値94.81750355533298距離0.2893475018198268 km
50325671221
存在メッシュ：50325671221値74.40875177843404距離0.14467375091535445 km
50325672111
存在メッシュ：50325672111値115.22625533027545距離0.4340212527104301 km
50325672121
存在メッシュ：50325672121値156.04375886829388距離0.7233687544075176 km
50325672112
存在メッシュ：50325672112値135.63500710172627距離0.578695003576282 km
50325671212
存在メッシュ：50325671212値54.0距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値57.0距離0.0 km
50325671221
存在メッシュ：50325671221値80.3822123024443距離0.14467375091535445 km
50325661434
存在メッシュ：50325661434

33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値58.0距離0.0 km
50325671221
存在メッシュ：50325671221値83.67988634372496距離0.14467375091535445 km
50325661434
存在メッシュ：50325661434値78.50868053742137距離0.11554053238239893 km
50325672211
存在メッシュ：50325672211値237.75920432234398距離1.0127162559357643 km
50325672122
存在メッシュ：50325672122値212.07931801013106距離0.8680425051979405 km
50325671222
存在メッシュ：50325671222値109.35977268551831距離0.2893475018198268 km
50325671221
存在メッシュ：50325671221値83.67988634372496距離0.14467375091535445 km
50325672111
存在メッシュ：50325672111値135.0396590248499距離0.4340212527104301 km
50325672121
存在メッシュ：50325672121値186.39943168858173距離0.7233687544075176 km
50325672112
存在メッシュ：50325672112値160.71954535978804距離0.578695003576282 km
50325671212
存在メッシュ：50325671212値58.0距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値68.0距離0.0 km
50325671221
存在メッシュ：50325671221値94.08535823336271距離0.14467375091535445 km
50325661434
存在メッシュ：503256614

33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値62.0距離0.0 km
50325671221
存在メッシュ：50325671221値84.43611122662286距離0.14467375091535445 km
50325661434
存在メッシュ：50325661434値79.9181103642734距離0.11554053238239893 km
50325672211
存在メッシュ：50325672211値219.0527785132058距離1.0127162559357643 km
50325672122
存在メッシュ：50325672122値196.61666731411452距離0.8680425051979405 km
50325671222
存在メッシュ：50325671222値106.8722224515581距離0.2893475018198268 km
50325671221
存在メッシュ：50325671221値84.43611122662286距離0.14467375091535445 km
50325672111
存在メッシュ：50325672111値129.30833367434252距離0.4340212527104301 km
50325672121
存在メッシュ：50325672121値174.18055610686613距離0.7233687544075176 km
50325672112
存在メッシュ：50325672112値151.74444489328852距離0.578695003576282 km
50325671212
存在メッシュ：50325671212値62.0距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値59.0距離0.0 km
50325671221
存在メッシュ：50325671221値81.57126852316878距離0.14467375091535445 km
50325661434
存在メッシュ：50325661434

33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値56.0距離0.0 km
50325671221
存在メッシュ：50325671221値78.03063933698509距離0.14467375091535445 km
50325661434
50325661434 is kddi none!
50325672211
存在メッシュ：50325672211値210.21447528706352距離1.0127162559357643 km
50325672122
存在メッシュ：50325672122値188.18383597711244距離0.8680425051979405 km
50325671222
存在メッシュ：50325671222値100.06127867231308距離0.2893475018198268 km
50325671221
存在メッシュ：50325671221値78.03063933698509距離0.14467375091535445 km
50325672111
存在メッシュ：50325672111値122.09191800552912距離0.4340212527104301 km
50325672121
存在メッシュ：50325672121値166.1531966591517距離0.7233687544075176 km
50325672112
存在メッシュ：50325672112値144.12255733497605距離0.578695003576282 km
50325671212
存在メッシュ：50325671212値56.0距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値60.0距離0.0 km
50325671221
存在メッシュ：50325671221値82.43611122662286距離0.14467375091535445 km
50325661434
50325661434 is kddi none!
50325672211
存在メッシュ：503256

...

33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値57.0距離0.0 km
50325671221
存在メッシュ：50325671221値80.92284148862798距離0.14467375091535445 km
50325661434
存在メッシュ：50325661434値76.10545502696623距離0.11554053238239893 km
50325672211
存在メッシュ：50325672211値224.45989034239412距離1.0127162559357643 km
50325672122
存在メッシュ：50325672122値200.5370488831221距離0.8680425051979405 km
50325671222
存在メッシュ：50325671222値104.84568297545653距離0.2893475018198268 km
50325671221
存在メッシュ：50325671221値80.92284148862798距離0.14467375091535445 km
50325672111
存在メッシュ：50325672111値128.76852445999174距離0.4340212527104301 km
50325672121
存在メッシュ：50325672121値176.61420741515246距離0.7233687544075176 km
50325672112
存在メッシュ：50325672112値152.69136594043414距離0.578695003576282 km
50325671212
存在メッシュ：50325671212値57.0距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値53.0距離0.0 km
50325671221
存在メッシュ：50325671221値73.27359448188813距離0.14467375091535445 km
50325661434
存在メッシュ：503256614

33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値60.0距離0.0 km
50325671221
50325671221 is kddi none!
50325661434
存在メッシュ：50325661434値74.57195722395728距離0.11554053238239893 km
50325672211
存在メッシュ：50325672211値187.72364517640227距離1.0127162559357643 km
50325672122
存在メッシュ：50325672122値169.47741016509312距離0.8680425051979405 km
50325671222
存在メッシュ：50325671222値96.49247006602617距離0.2893475018198268 km
50325671221
50325671221 is kddi none!
50325672111
存在メッシュ：50325672111値114.73870509660387距離0.4340212527104301 km
50325672121
存在メッシュ：50325672121値151.23117514715017距離0.7233687544075176 km
50325672112
存在メッシュ：50325672112値132.9849401240599距離0.578695003576282 km
50325671212
存在メッシュ：50325671212値60.0距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値45.0距離0.0 km
50325671221
50325671221 is kddi none!
50325661434
存在メッシュ：50325661434値64.42927629860972距離0.11554053238239893 km
50325672211
存在メッシュ：50325672211値215.2981935685364距離1.012716255

33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値59.0距離0.0 km
50325671221
存在メッシュ：50325671221値80.62516744734734距離0.14467375091535445 km
50325661434
存在メッシュ：50325661434値76.27046782098643距離0.11554053238239893 km
50325672211
存在メッシュ：50325672211値210.37617206092125距離1.0127162559357643 km
50325672122
存在メッシュ：50325672122値188.75100464011038距離0.8680425051979405 km
50325671222
存在メッシュ：50325671222値102.25033489306804距離0.2893475018198268 km
50325671221
存在メッシュ：50325671221値80.62516744734734距離0.14467375091535445 km
50325672111
存在メッシュ：50325672111値123.8755023367157距離0.4340212527104301 km
50325672121
存在メッシュ：50325672121値167.12583721143724距離0.7233687544075176 km
50325672112
存在メッシュ：50325672112値145.5006697766636距離0.578695003576282 km
50325671212
存在メッシュ：50325671212値59.0距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値58.0距離0.0 km
50325671221
存在メッシュ：50325671221値80.97674041280654距離0.14467375091535445 km
50325661434
存在メッシュ：5032566143

33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値65.0距離0.0 km
50325671221
50325671221 is kddi none!
50325661434
存在メッシュ：50325661434値80.65136146276895距離0.11554053238239893 km
50325672211
存在メッシュ：50325672211値202.18465593020986距離1.0127162559357643 km
50325672122
存在メッシュ：50325672122値182.58684795510004距離0.8680425051979405 km
50325671222
存在メッシュ：50325671222値104.19561599684292距離0.2893475018198268 km
50325671221
50325671221 is kddi none!
50325672111
存在メッシュ：50325672111値123.7934239926486距離0.4340212527104301 km
50325672121
存在メッシュ：50325672121値162.989039972865距離0.7233687544075176 km
50325672112
存在メッシュ：50325672112値143.3912319851014距離0.578695003576282 km
50325671212
存在メッシュ：50325671212値65.0距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値53.0距離0.0 km
50325671221
50325671221 is kddi none!
50325661434
存在メッシュ：50325661434値72.64515714637204距離0.11554053238239893 km
50325672211
存在メッシュ：50325672211値225.1903957192979距離1.01271625593

33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値41.0距離0.0 km
50325671221
存在メッシュ：50325671221値63.436111226622856距離0.14467375091535445 km
50325661434
存在メッシュ：50325661434値58.9181103642734距離0.11554053238239893 km
50325672211
存在メッシュ：50325672211値198.0527785132058距離1.0127162559357643 km
50325672122
存在メッシュ：50325672122値175.61666731411452距離0.8680425051979405 km
50325671222
存在メッシュ：50325671222値85.8722224515581距離0.2893475018198268 km
50325671221
存在メッシュ：50325671221値63.436111226622856距離0.14467375091535445 km
50325672111
存在メッシュ：50325672111値108.30833367434253距離0.4340212527104301 km
50325672121
存在メッシュ：50325672121値153.18055610686613距離0.7233687544075176 km
50325672112
存在メッシュ：50325672112値130.74444489328852距離0.578695003576282 km
50325671212
存在メッシュ：50325671212値41.0距離0.0 km
33.8057291665:33.8088541665:132.771093750:132.771093750
50325671212
存在メッシュ：50325671212値43.0距離0.0 km
50325671221
存在メッシュ：50325671221値65.57126852316878距離0.14467375091535445 km
50325661434
存在メッシュ：5032566143

In [143]:
connection.close()

In [123]:
#CreateGeojsonFile